In [1]:
!pip install geocoder

!pip install -c conda-forge geopy --yes  
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!pip install -c conda-forge folium=0.5.0 --yes  
###import folium # map rendering library


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: --yes

Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: --yes


In [2]:
!pip install folium

In [3]:
import folium

In [4]:
from bs4 import BeautifulSoup
import requests
import re 
import pandas as pd, numpy as np
import geocoder
import io

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


## References used:
#Lab: New York, FourSquare API and clustering
#https://stackoverflow.com/questions/50355577/scraping-wikipedia-tables-with-python-selectively
#https://stackoverflow.com/questions/42776834/get-the-title-of-a-link-using-beautifulsoup
#https://stackoverflow.com/questions/46275765/pandas-merge-row-data-with-multiple-values-to-python-list-for-a-column
#https://www.geeksforgeeks.org/python-program-split-join-string/
#https://stackoverflow.com/questions/32400867/pandas-read-csv-from-url

In [5]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

res = requests.get(URL).content   
soup = BeautifulSoup(res,'lxml')

all_items = []
# Find all with "tr"
for items in soup.find('table', class_='wikitable').find_all('tr')[1::1]:
    # Find items in the table
    data = items.find_all(['th','td'])

    # Get the raw strings
    postcode = str(data[0].contents[0] )
    borough_1 = str(data[1].contents[0] )
    neighbourhood_1 = str(data[2].contents[0] )

    # Remove links in Borough, if any
    if re.search(   '</a>'  , borough_1 ) :
      borough = re.split('>|<|</a>',  borough_1  ) [-3:-2][0]  
    else : 
      borough = borough_1
    
    # Remove links in Neighbourhood, if any
    if re.search(   '</a>'  , neighbourhood_1 ) :
      neighbourhood = re.split('>|<|</a>',  neighbourhood_1  ) [-3:-2][0] 
    else :
      neighbourhood = neighbourhood_1
    
    # Remove Extra line breaks if they appear
    postcode = re.sub(r'\n', '', postcode)   
    borough = re.sub(r'\n', '', borough)   
    neighbourhood = re.sub(r'(\n)', r'', neighbourhood) 
    if neighbourhood == "Not assigned" :
      neighbourhood = borough

    # Create the new row, and add to the list
    if borough != "Not assigned" :
      new_row = [postcode, borough, neighbourhood]
      all_items.append(new_row)

In [6]:
# Creating a dataframe object from list of lists
all_items_df = pd.DataFrame(all_items) 
all_items_df.columns =['PostalCode', 'Borough', 'Neighbourhood'] 

# For PostCode with multiple Neighborhood, collapse them into a list
grouped_df = all_items_df.groupby(['PostalCode', 'Borough']).agg(lambda x: ', '.join( tuple(x)  )  ).reset_index()    
 
# Sort by postal code
Toronto_df = grouped_df.sort_values(['PostalCode'])

## Toronto_df.head()
## Toronto_df.shape

In [7]:
# Load data from the csv file
geo_URL="http://cocl.us/Geospatial_data"
geo_bytes=requests.get(geo_URL).content
geo_data=pd.read_csv(io.StringIO(geo_bytes.decode('utf-8')))
## print( geo_data.head() )

In [8]:
Toronto_LatLong = pd.merge(Toronto_df, geo_data, left_on='PostalCode', right_on='Postal Code')[['PostalCode', 'Borough', 'Neighbourhood','Latitude','Longitude']]

##Toronto_LatLong.head()

In [9]:
address = 'Toronto, ON'  

geolocator = Nominatim(user_agent="ON_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
## print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [10]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[  latitude, longitude ], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_LatLong['Latitude'], Toronto_LatLong['Longitude'], Toronto_LatLong['Borough'], Toronto_LatLong['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
### map_Toronto

In [11]:
CLIENT_ID = 'WLCPKLDTCHGABWDC4ZYERUR44LCZMHSTYRR2CHRYRWIFUURJ' # your Foursquare ID
CLIENT_SECRET = 'ZZXXL41D0BKQKTXBFQFMBZRZ34DN2YOFGIFOWFHPV533RVUM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

##print('Your credentails:')
##print('CLIENT_ID: ' + CLIENT_ID)
##print('CLIENT_SECRET:' + CLIENT_SECRET)


LIMIT = 100 #100 # limit of number of venues returned by Foursquare API

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        ##### print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        ##### print(type(results))
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:

toronto_venues = getNearbyVenues(names=Toronto_LatLong['Neighbourhood'],
                                   latitudes=Toronto_LatLong['Latitude'],
                                   longitudes=Toronto_LatLong['Longitude']
                                  )

In [14]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
## toronto_grouped

In [16]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood'].head() :
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge   0.2
1               Skating Rink   0.2
2             Breakfast Spot   0.2
3  Latin American Restaurant   0.2
4             Clothing Store   0.2


----Alderwood / Long Branch----
                venue  freq
0         Pizza Place   0.2
1                 Gym   0.1
2  Athletics & Sports   0.1
3                Pool   0.1
4         Coffee Shop   0.1


----Bathurst Manor / Wilson Heights / Downsview North----
                       venue  freq
0                       Bank  0.10
1                Coffee Shop  0.10
2             Ice Cream Shop  0.05
3  Middle Eastern Restaurant  0.05
4                 Restaurant  0.05


----Bayview Village----
                 venue  freq
0                 Bank  0.25
1   Chinese Restaurant  0.25
2  Japanese Restaurant  0.25
3                 Café  0.25
4          Yoga Studio  0.00


----Bedford Park / Lawrence Manor East----
                venue  freq
0          Restaurant  0.08

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [18]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Latin American Restaurant,Skating Rink,Lounge,Breakfast Spot,Clothing Store
1,Alderwood / Long Branch,Pizza Place,Gym,Coffee Shop,Pharmacy,Skating Rink
2,Bathurst Manor / Wilson Heights / Downsview North,Bank,Coffee Shop,Shopping Mall,Ice Cream Shop,Sushi Restaurant
3,Bayview Village,Café,Chinese Restaurant,Bank,Japanese Restaurant,Eastern European Restaurant
4,Bedford Park / Lawrence Manor East,Coffee Shop,Restaurant,Sandwich Place,Italian Restaurant,Sushi Restaurant


In [19]:

toronto_merged = Toronto_LatLong
toronto_merged.columns = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']

## Introduction

The purpose of this study is to decide which is the best neighborhood for a new Italian restaurant in Toronto. We are advising a chef from another part of Canada on where is the best place in Toronto to open a new restaurant. The criteria for deciding where to place the new Italian restaurant includes two main points:

1. The new Italian restaurant should not be in the same neighborhood as an existing Italian restaurant

2. The neighborhood chosen should have a lot in common with the neighborhoods that already have Italian restaurants. For example, if we find that it is common for Italian restaurants to be located in neighborhoods that also have Mexican restaurants, then we would like to find a neighborhood with a Mexican restaurant to open our new Italian restaurant.

These two criteria ensure that the new Italian restaurant will be in a location that is similar to the neighborhoods that already have Italian restaurants, but won't be too close to an existing Italian restaurant.

## Data

The data for this study comes from Wikipedia, the Python _geocoder_ package, and FourSquare. Wikipedia provides names and postcodes for 10 boroughs and 103 neighborhoods in Toronto. The _geocoder_ package was used to create the csv file of longitude and latitude values for each Toronto neighborhood, and this csv file was provided by Coursera. Foursquare will be used to look up each neighborhood by it's coordinates and provide a list of all businesses in that area. The Foursquare data will let us see which neighborhoods already have at least one Italian restaurant, and also what other types of businesses may be present.

#### Wikipedia data

This shows the boroughs and neighborhood names available from Wikipedia. The PostalCode can be joined to the _geocoder_ csv output to find longitude and latitude, which allows for _folium_ map visualizations

In [20]:
# Check how many boroughs and neighbourhoods there are
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Toronto_LatLong['Borough'].unique()),
        Toronto_LatLong.shape[0]
    )
)
print("Boroughs are ", Toronto_LatLong['Borough'].unique())
Toronto_df.head()

The dataframe has 10 boroughs and 103 neighborhoods.
Boroughs are  ['Scarborough' 'North York' 'East York' 'East Toronto' 'Central Toronto'
 'Downtown Toronto' 'York' 'West Toronto' 'Mississauga' 'Etobicoke']


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### _geocoder_ data
This is the Latitude and Longitude values for map plotting

In [21]:
 geo_data.head() 

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Foursquare
These are the top 10 venue categories in Toronto. Italian restaurants are already in the top 10, so we need to find neighborhoods that don't have one yet. Also, an interesting finding will be to see how Pizza Place trends with Italian restaurant. We will have to decide whether our new restaurant should be in a location that already has a pizza place.

In [22]:
 toronto_venues['Venue Category'].value_counts().head(10)

Coffee Shop            180
Café                    98
Restaurant              74
Park                    52
Italian Restaurant      50
Pizza Place             48
Hotel                   41
Bakery                  39
Sandwich Place          38
Japanese Restaurant     38
Name: Venue Category, dtype: int64

## Methodology

We will use FourSquare data to find which neighborhoods meet the two main criteria of not already having an Italian restaurant in the top 100 results listed, but being similar to neighborhoods that do have one.

1. The first step is to identify which restaurants already have an Italian restaurant in their first 100 FourSquare results. This is straightforward, and can be checked using the one-hot encoded dataset.

2. The second step is to use cosine similarity on the one-hot vectors for each of about 100 neighborhoods to compare each neighborood without an Italian restaurant to each neighborhood that already has an Italian restaurant.

In [23]:
toronto_Italian_labels = toronto_grouped.copy()

toronto_Italian_labels['Italian_Places'] = np.where(  (toronto_Italian_labels['Italian Restaurant']>0) & (toronto_Italian_labels['Pizza Place'] ==0) , 1, 
                         np.where(  (toronto_Italian_labels['Pizza Place'] >0) & (toronto_Italian_labels['Italian Restaurant'] ==0)  , 2,
                           np.where( (toronto_Italian_labels['Italian Restaurant'] >0) & (toronto_Italian_labels['Pizza Place'] >0)  , 3,0 )))  #and (toronto_Italian_labels['Pizza Place'] >0)

toronto_Italian_labels['Italian_Labels'] = np.where(  (toronto_Italian_labels['Italian_Places']==1) , "Italian Restaurant, no Pizza Place", 
                         np.where(  (toronto_Italian_labels['Italian_Places'] ==2 )   , "Pizza Place but no Italian Restaurant",
                           np.where( (toronto_Italian_labels['Italian_Places'] ==3 )  , "Italian Restaurant but no Pizza Place", "Neither Italian Restaurant or Pizza Place" )))  #and (toronto_Italian_labels['Pizza Place'] >0)


toronto_Italian_labels[ ["Neighborhood", "Italian Restaurant", "Pizza Place", 'Italian_Places', 'Italian_Labels'] ].head(10)    #toronto_merged
#print(toronto_grouped.columns )
# print(  len(toronto_grouped["Neighborhood"])   )
# print(  len(toronto_grouped["Neighborhood"].unique()  )   )

,Neighborhood,Italian Restaurant,Pizza Place,Italian_Places,Italian_Labels
0,Agincourt,0.000000,0.000000,0,Neither Italian Restaurant or Pizza Place
1,Alderwood / Long Branch,0.000000,0.200000,2,Pizza Place but no Italian Restaurant
2,Bathurst Manor / Wilson Heights / Downsview North,0.000000,0.050000,2,Pizza Place but no Italian Restaurant
3,Bayview Village,0.000000,0.000000,0,Neither Italian Restaurant or Pizza Place
4,Bedford Park / Lawrence Manor East,0.083333,0.041667,3,Italian Restaurant but no Pizza Place
5,Berczy Park,0.035714,0.000000,1,"Italian Restaurant, no Pizza Place"
6,Birch Cliff / Cliffside West,0.000000,0.000000,0,Neither Italian Restaurant or Pizza Place
7,Brockton / Parkdale Village / Exhibition Place,0.043478,0.000000,1,"Italian Restaurant, no Pizza Place"
8,Business reply mail Processing CentrE,0.000000,0.055556,2,Pizza Place but no Italian Restaurant
9,CN Tower / King and Spadina / Railway Lands / ...,0.000000,0.000000,0,Neither Italian Restaurant or Pizza Place


In [24]:
toronto_Italian_labels['Italian_Labels'].value_counts()

Neither Italian Restaurant or Pizza Place    51
Pizza Place but no Italian Restaurant        19
Italian Restaurant but no Pizza Place        14
Italian Restaurant, no Pizza Place           11
Name: Italian_Labels, dtype: int64

In [29]:
toronto_Italian_labels['Italian_Places'].value_counts()

0    51
2    19
3    14
1    11
Name: Italian_Places, dtype: int64

## Results

In [25]:

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged_Italian = toronto_merged.join(toronto_Italian_labels[ ['Neighborhood', 'Italian_Places'] ].set_index('Neighborhood'), on='Neighborhood')

In [30]:

toronto_merged_Italian = toronto_merged_Italian.dropna()
toronto_merged_Italian['Italian_Places'] = toronto_merged_Italian[['Italian_Places']].astype(int)

toronto_merged_Italian.head() # check the last columns!


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Italian_Places
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,0
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,0
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711,0
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0


In [31]:
print(toronto_merged_Italian.shape )
print(toronto_merged_Italian.head() )

(100, 6)
  PostalCode      Borough                              Neighborhood  \
0        M1B  Scarborough                           Malvern / Rouge   
1        M1C  Scarborough  Rouge Hill / Port Union / Highland Creek   
2        M1E  Scarborough       Guildwood / Morningside / West Hill   
3        M1G  Scarborough                                    Woburn   
4        M1H  Scarborough                                 Cedarbrae   

    Latitude  Longitude  Italian_Places  
0  43.806686 -79.194353               0  
1  43.784535 -79.160497               0  
2  43.763573 -79.188711               0  
3  43.770992 -79.216917               0  
4  43.773136 -79.239476               0  


In [32]:
# create map
map_Italian = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(4)
ys = [i + x + (i*x)**2 for i in range(4)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_Italian['Latitude'], toronto_merged_Italian['Longitude'], toronto_merged_Italian['Neighborhood']
                                  , toronto_merged_Italian['Italian_Places']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],     #[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_Italian)
       
map_Italian

In [64]:
print(colors_array)
print(rainbow)

[[5.00000000e-01 0.00000000e+00 1.00000000e+00 1.00000000e+00]
 [1.66666667e-01 8.66025404e-01 8.66025404e-01 1.00000000e+00]
 [8.33333333e-01 8.66025404e-01 5.00000000e-01 1.00000000e+00]
 [1.00000000e+00 1.22464680e-16 6.12323400e-17 1.00000000e+00]]
['#8000ff', '#2adddd', '#d4dd80', '#ff0000']


## Discussion

## Conclusion